# Grad-CAM Refinement & Testing (Week 2, Day 4)

## Notebook Purpose

This notebook (`04_gradcam_refinement.ipynb`) extends the PneumoDetect explainability workflow by refining and testing the Grad-CAM implementation introduced in Week 2 Day 3.

### Objectives
- Encapsulate Grad-CAM inference in a reusable `generate_cam(image_path, model_path)` function.
- Validate reliability and reproducibility with unit and integration tests.
- Enhance color rendering using OpenCV’s `COLORMAP_JET`.
- Ensure Grad-CAM overlays render consistently across different checkpoints.
- Summarize and visualize Grad-CAM results for misclassified examples.

### Technical Summary
- **Module Updated:** `src/gradcam.py`
- **Tests Added:** `tests/test_gradcam_refinement.py`, `tests/test_gradcam_refinement_integration.py`
- **Outputs Saved To:** `reports/week2_gradcam_refinement/`

### Deliverables
- Passing test suite (`pytest`): verifies shape, normalization (0–1), and overlay integrity.
- Reliable Grad-CAM visualizations confirming robust explainability.

In [2]:
# Add script to include project root in sys.path

import sys
from pathlib import Path

# Add project root (one level up from notebooks/)
project_root = Path(__file__).resolve().parent.parent if "__file__" in globals() else Path.cwd().parent
sys.path.append(str(project_root))

In [3]:
# Import necessary libraries

from pathlib import Path
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from src.gradcam import GradCAM, generate_cam

In [4]:
# ------------------------------------------------------------
# 1. Setup & Directories
# ------------------------------------------------------------
project_root = Path.cwd().parents[0] if Path.cwd().name == 'notebooks' else Path.cwd()
output_dir = project_root / 'reports' / 'week2_gradcam_refinement'
output_dir.mkdir(parents=True, exist_ok=True)

model_path = project_root / 'saved_models' / 'resnet50_baseline.pt'
finetuned_model_path = project_root / 'saved_models' / 'resnet50_finetuned.pt'
image_dir = project_root / 'data' / 'rsna_subset' / 'train_images'

print(f"Using models from: {model_path.parent}")
print(f"Saving refined outputs to: {output_dir}")

Using models from: /Users/adrianadewunmi/VSCODE/AI-Assisted-Pneumonia-Detection-Project/saved_models
Saving refined outputs to: /Users/adrianadewunmi/VSCODE/AI-Assisted-Pneumonia-Detection-Project/reports/week2_gradcam_refinement


In [5]:
# ------------------------------------------------------------
# 2. Generate Grad-CAM Heatmap Wrapper
# ------------------------------------------------------------
def generate_cam(image_path: Path, model_path: Path) -> np.ndarray:
    """
    Generate a Grad-CAM heatmap for a given image using a model checkpoint.

    Args:
        image_path (Path): Path to an input image.
        model_path (Path): Path to a trained model checkpoint.

    Returns:
        np.ndarray: Normalized Grad-CAM heatmap (H x W, range 0–1).
    """
    if not image_path.exists():
        raise FileNotFoundError(f"Image file not found: {image_path}")
    if not model_path.exists():
        raise FileNotFoundError(f"Model checkpoint not found: {model_path}")

    from src.model import build_resnet50_baseline
    model = build_resnet50_baseline()
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
    model.eval()

    cam = GradCAM(model, target_layer_name='layer4')

    img = Image.open(image_path).convert('RGB')
    tensor = torch.from_numpy(np.array(img)).permute(2, 0, 1).unsqueeze(0).float() / 255.0

    heatmap = cam.generate(tensor)
    heatmap = heatmap.astype(np.float32) / 255.0
    return heatmap

In [6]:
# ------------------------------------------------------------
# 3. Visual Comparison Utility
# ------------------------------------------------------------
def compare_models(image_path: Path, model_paths: list):
    """Compare Grad-CAM results from multiple model checkpoints."""
    results = []
    for path in model_paths:
        heatmap = generate_cam(image_path, path)
        img = cv2.imread(str(image_path))
        overlay = GradCAM.overlay_heatmap(heatmap, img)

        out_name = f"{image_path.stem}_{path.stem}_overlay.png"
        out_path = output_dir / out_name
        cv2.imwrite(str(out_path), overlay)

        results.append((path.stem, heatmap, overlay))
        print(f"Saved overlay: {out_path.name}")

    return results